In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import sqlite3

In [2]:

app = FastAPI()
Product = []

class Item(BaseModel):
    Pname: str
    price: float;
    qty: int


def init():
    db = sqlite3.connect('product.db')
    db.execute('''
    CREATE TABLE IF NOT EXISTS PRODUCT
    (ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME TEXT UNIQUE NOT NULL,
    PRICE REAL NOT NULL,
    QTY INT NOT NULL);''')
    print("create successfully")
    db.close()

init()
@app.get('/')
async def home():
    return {"message":"Hello world"}


@app.post('/Product')
async def addproduct(item: Item):
    db = sqlite3.connect('product.db')
    data = (item.Pname,item.price,item.qty)
    cur = db.cursor()
    sql = """INSERT OR IGNORE INTO PRODUCT(NAME, PRICE, QTY) VALUES(?,?,?)"""
    cur.execute(sql, data)
    db.commit()
    return {"message":"insert completed "+str(cur.lastrowid)}

@app.get('/Product')
async def gettallproduct():
    db = sqlite3.connect('product.db')
    sql = "select * from PRODUCT;"
    cur = db.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    return {"message" : "get all data succesed", "Data":rows}


@app.put('/Product/{ID}')
async def updateproduct(item:Item , ID:int):
    db = sqlite3.connect('product.db')
    data = (item.Pname,item.price,item.qty)
    cur = db.cursor()
    sql = """UPDATE PRODUCT
        SET NAME = {item.Pname}, PRICE = {item.price}, QTY = {item.qty}, WHERE ID = `{ID}`;"""
    cur.execute(sql, data)
    db.commit()
    return {"message","UPDATE completed "+str(cur.lastrowid)}

@app.delete('/Product/{ID}')
async def deleteproduct(ID:int):
    db = sqlite3.connect('product.db')
    cur = db.cursor()
    sql = """DELETE FROM PRODUCT WHERE ID=(%s));"""%ID
    cur.execute(sql)
    db.commit()
    return {"message","DELETE completed "+str(cur.lastrowid)}

@app.get('/test/{id}')
async def test_opt(id:int, otp: str='EQUAL'):
    option = []
    if otp=='MORE':
        option.append('set to more')
    elif otp=='LESS':
        option.append('set to less')
    else:
        option.append('set to equal')
    return {
        "message": "test and point with get option",
        "option": option
    }




create successfully


In [3]:
init()
nest_asyncio.apply()
uvicorn.run(app,port=8080)

INFO:     Started server process [64914]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


create successfully
INFO:     127.0.0.1:63034 - "GET /test/1?otp=LESS HTTP/1.1" 200 OK
INFO:     127.0.0.1:63037 - "GET /Product HTTP/1.1" 200 OK
